In [33]:
## Pyro GP tutorial used as starting point:
## https://pyro.ai/examples/gp.html

import matplotlib.pyplot as plt
import numpy as np
import torch
import pyro
import pyro.contrib.gp as gp
import pyro.distributions as dist
import arviz

# Partition observations
X = np.asarray([x / 29 for x in range(1, 31)])
np.random.shuffle(X)
Y = 6 * np.square(X) - np.square(np.sin(6 * np.pi * X)) - 5 * np.power(X, 4) + 3 / 2 + np.random.normal(0.0, 0.1, 30)
Xtrain, Xtest, Ytrain, Ytest = torch.tensor(X[10:]), torch.tensor(X[:10]), torch.tensor(Y[10:]), torch.tensor(Y[:10])

### Selecting a suitable model

In [34]:
# We chose a GP regression model and the Matern 3/2 kernel. In this setup, we have three hyper-parameters.
# I) The variance of the kernel, II) the lengthscale of the kernel, and III) the gaussian noise of the model.
# We chose to let the gaussian noise be fixed and equal to the noise of our data, while keeping the variance
# and lengthscale of the kernel variable. The prior distrubition we chose is a multivariate normal
# distribution (i.e. we consider the variance and lengthscale as normally distributed), with mean and variance
# based on what seems reasonable for the Matern 3/2 kernel, based on the lecture slides.

# Pick prior distribution
prior = dist.MultivariateNormal(torch.tensor([1.5,1]), torch.eye(2))

# Define model
def model(xs, ys, theta):
    kernel = gp.kernels.Matern32(input_dim=1, variance=theta[0], lengthscale=theta[1])
    return gp.models.GPRegression(xs, ys, kernel, noise=torch.tensor(0.01))

# Computes log-likelihood
def logLikelihood(xs, ys, theta=prior.sample()):
    # solution based on lecture notes section 6.3
    kernel = gp.kernels.Matern32(input_dim=1, variance=theta[0], lengthscale=theta[1])
    t1 = 0.5 * torch.transpose(ys, 0, 0) * torch.linalg.inv(kernel.forward(xs)) * ys
    t2 = 0.5 * torch.log(torch.linalg.det(kernel.forward(xs)))
    t3 = 15.0 * torch.log(2 * torch.tensor(np.pi))
    return - t1 - t2 - t3

### Using NUTS

In [35]:
# Model is GP model from pyro
W = 100 # Number of warmup steps
C = 1 # Number of chains
S = 500 # Number of samples used in prediction

gp_model_nuts = model(Xtrain, Ytrain, prior.sample())
nuts_kernel = pyro.infer.NUTS(model, jit_compile=True)
mcmc = pyro.infer.MCMC(nuts_kernel, S, num_chains=C, warmup_steps=W)
mcmc.run(Xtrain, Ytrain, prior.sample())

Sample: 100%|██████████| 600/600 [00:00, 55823.57it/s, step size=1.00e+00, acc. prob=1.000]


#### Checking quality of samples using arviz

In [36]:
posterior_samples = mcmc.get_samples()
data = arviz.from_pyro(mcmc)
print(data)
summary = arviz.summary(data)
print(summary)
arviz.plot_trace(data)
plt.show()
# Maybe use this: arviz.rcParams['plot.max_subplots'] = 18
# arviz.plot_posterior(data, var_names=['w3', 'b3']) # TODO: Change var names

/Users/vic/Desktop/Computer Science/2/PML/Final Project/PMLFinalProject/venv/lib/python3.11/site-packages/arviz/stats/stats.py:1354: UserWarning: Selecting first found group: sample_stats
  warnings.warn(f"Selecting first found group: {data.groups()[0]}")
arviz - WARNING - Shape validation failed: input_shape: (1, 500), minimum_shape: (chains=2, draws=4)


Inference data with groups:
	> sample_stats
           mean   sd  hdi_3%  hdi_97%  mcse_mean  mcse_sd  ess_bulk  ess_tail  \
diverging   0.0  0.0     0.0      0.0        0.0      0.0     500.0     500.0   

           r_hat  
diverging    NaN  
